# Hubbard POD

In [ ]:
import rioxarray
import xarray
import numpy as np
import scipy
import matplotlib.pyplot as plt
import geopandas as gpd
import cartopy

## Load datacube

This file is on the drive

In [ ]:
xds_rio = rioxarray.open_rasterio("./hubbard_inversion_2015-10-01_2023-01-01_xform.nc")
xds = xarray.load_dataset("./hubbard_inversion_2015-10-01_2023-01-01_xform.nc")

## Crop to shapefile extent

In [ ]:
bound = gpd.read_file("./hubbard_outline.gpkg")
xds_clip = xds_rio.rio.clip(bound.geometry.values, bound.crs)
mask = np.isnan(xds_clip["vx"][0,:,:]) # mask for later
xds_clip = xds_clip.fillna(0)

## Interpolate onto regular times

In [ ]:
# Interpolate to daily velocities
s_in_day = 60 * 60 * 24
epoch = 2015
epoch = np.datetime64(str(epoch), "s")

# datetime64 bounds and steps
t0 = np.datetime64("2017-06-01", "ns") #np.min(xds["time"].to_numpy())
t1 = np.datetime64("2022-06-01", "ns") #np.max(xds["time"].to_numpy())
dt = np.timedelta64(1, "D")
nstep = int((t1 - t0) / dt)
tsteps = np.array([t0 + dt * i for i in range(nstep)])

v = np.sqrt(xds_clip["vx"]**2 + xds_clip["vy"]**2)
spline = scipy.interpolate.CubicSpline(xds["time"], v, axis=0)

vdaily = spline(tsteps)

## Remove mean in time

In [ ]:
vmean = np.mean(vdaily, axis=0)
vdaily_demean = vdaily - vmean

## Flatten timesteps

In [ ]:
vdaily_flat = np.zeros((vdaily_demean.shape[1]*vdaily_demean.shape[2], vdaily_demean.shape[0]))

for i in range(vdaily.shape[0]):
    vdaily_flat[:,i] = vdaily_demean[i,:,:].flatten()

## Do SVD

In [ ]:
U, S, Vh = scipy.linalg.svd(vdaily_flat)

In [ ]:
prj = cartopy.crs.epsg("3338")
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=prj)
grid_extent = (vel["x"].min(), vel["x"].max(), vel["y"].min(), vel["y"].max())


## Rebuild spatial modes and plot temporal weights

In [ ]:
for i in range(10):
    m = U[:,i].reshape(v.shape[1], v.shape[2])
    tw = Vh[i,:]

    fig, axs = plt.subplots(1, 2, figsize=(10, 3), gridspec_kw={'width_ratios': [1, 4]}, layout="constrained")
    m[mask] = np.nan
    im = axs[0].imshow(m, cmap="jet")
    axs[0].axis('off')
    cbar = plt.colorbar(im, ax=axs[0], location="left")

    plt.figure()
    axs[1].plot(tsteps, tw)
    axs[1].grid(True, linestyle="--")

    fig.suptitle("Mode %d" % i)
    
    fig.savefig("modes/mode%d.png" % i, bbox_inches="tight", dpi=300)
    fig.show()